# Opioid Data - LSTM
HW #2 Part 2 - Timeseries.  
Use all rows per patient from about 30 consecutive days.
Standardize all of it.
Train LSTM classifier.

## Load data structures and scale features

In [1]:
from os import listdir
import numpy as np
import pandas as pd
import time

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.preprocessing.sequence import pad_sequences

## From data file to Pandas dataframe

In [2]:
pathR='data/ChunkedData_R/'
pathN='data/chunkedData_NR/'

In [3]:
def initialize_globals():
    global patient_list, labels_list, features_df,UNIFORM_NUM_DAYS
    UNIFORM_NUM_DAYS = 29  # most daily records of any patient
    patient_list = [] # list of 40 (cohort,patient_ID,first_row,last_row+1)
    labels_list = [] # list of 1004 (cohort,patient_ID,date)
    features_df = pd.DataFrame()  # 1004 rows of 259 numeric values

In [4]:
# Read one CSV file. 
# Load global lists
def load_patient (filepath,cohort,patient_ID):
    global patient_list
    global labels_df
    global features_df
    one_patient = pd.read_csv(filepath)
    rows,cols = one_patient.shape
    features_df = features_df.append(one_patient)
    first_insert = len(labels_list)
    last_insert = first_insert + rows
    for rec in range(0,rows):
        one_label=(cohort,patient_ID,one_patient.loc[rec]['Date'])
        labels_list.append(one_label)
    patient_list.append([cohort,patient_ID,first_insert,last_insert])

# Read directory of CSV files (R or NR). 
# Given directory, load all the patients in that directory.
# We use filenames as patient names.
def load_cohort (cohort,directory):
    file_names = listdir(directory)
    for fp in file_names:
        dfp = directory+fp
        one_name = fp.split('.')[0]  # strip away .csv suffix
        one_name = one_name[6:]    # strip away Daily_ prefix
        one_patient = load_patient(dfp,cohort,one_name)

# Standardize features by shifting the mean to zero and scaling to unit variance.
# Subtract the mean and divide by the std.dev: z = (x - u) / s
def scale_features(df):
    scaled = StandardScaler().fit_transform(df.values)
    scaled_df = pd.DataFrame(scaled, index=df.index, columns=df.columns)
    return scaled_df

In [5]:
initialize_globals()
load_cohort('R',pathR)
load_cohort('N',pathN)
features_df = features_df.drop('Date',axis=1) 
scaled_features = scale_features(features_df)
features_df = None
print("Patients:",len(patient_list))
print("Label rows:",len(labels_list))
print("Feature rows:",len(scaled_features))

Patients: 40
Label rows: 1004
Feature rows: 1004


## Prepare numpy arrays as required by tensorflow

In [6]:
# Create numpy array of 40 1 or 0
def make_labels():
    # rely on global patient_list:
    # list of 40 (cohort,patient_ID,first_row,last_row+1)
    labels=[]
    for p in patient_list:
        cohort = p[0]
        y = 0
        if cohort=='R':
            y = 1
        labels.append(y)
        ary = np.array(labels).astype(np.float32)
    return ary

def make_data():
    patients=[]
    for p in patient_list:
        days=[]
        first = p[2]
        last = p[3]
        for r in range(first,last):
            day = scaled_features.iloc[r].values.tolist()
            days.append(day)
        patients.append(days)
    # The tensor data structure has fixed size for each dimension.
    # Thus we pad every patient to the maximum = UNIFORM_NUM_DAYS = 29.
    # The alternative is RaggedTensor which is slow and not fully supported.
    padded = pad_sequences(patients, padding="post")  
    ary = np.array(padded).astype(np.float32)
    return ary
        
X_all = make_data()
y_all = make_labels()
X_all,y_all = shuffle(X_all,y_all) # avoid training on all the R, then all the NR
print("y values, X values:",len(y_all),len(X_all))
UNIFORM_NUM_PATIENTS=len(y_all)
UNIFORM_NUM_DAYS=len(X_all[0])
UNIFORM_NUM_FEATURES=len(X_all[0][0])
print("Patients, Days, Features:",UNIFORM_NUM_PATIENTS,UNIFORM_NUM_DAYS,UNIFORM_NUM_FEATURES)

y values, X values: 40 40
Patients, Days, Features: 40 29 259


## LSTM

In [7]:
NEURONS=32
def build_model():
    # We padded each patient to 29 days. Pad value was zero. Here, mask thoses days.
    embed_layer = Masking(mask_value=0.,input_shape=(UNIFORM_NUM_DAYS,UNIFORM_NUM_FEATURES))
    rnn1_layer = LSTM(NEURONS, return_sequences=True) 
    rnn2_layer = LSTM(NEURONS, return_sequences=False)
    dense1_layer = Dense(NEURONS)
    dense2_layer = Dense(NEURONS)
    output_layer = Dense(1, activation="sigmoid")
    ann = keras.models.Sequential()
    ann.add(embed_layer)
    ann.add(rnn1_layer)
    ann.add(rnn2_layer)
    ann.add(dense1_layer)
    ann.add(dense2_layer)
    ann.add(output_layer)
    compiled = compile_model(ann)
    return compiled

# Default weight initializers.
# Keras Dense default = Glorot aka Xavier uniform initializer

def compile_model(model):
    bc=tf.keras.losses.BinaryCrossentropy(from_logits=False)
    model.compile(loss=bc, optimizer="adam", metrics=["accuracy"])
    return model

In [8]:
lstm = build_model()
lstm = compile_model(lstm)
lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 29, 259)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 29, 32)            37376     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 47,841
Trainable params: 47,841
Non-trainable params: 0
____________________________________________________

In [14]:
#lstm.fit(X_all,y_all)
splitter=LeaveOneOut()
EPOCHS=5
BATCH = UNIFORM_NUM_PATIENTS//4
print("Num splits:",splitter.get_n_splits(X_all))
truth = []
predicted = []
for train_index,test_index in splitter.split(X_all,y_all):
    lstm = build_model()
    lstm = compile_model(lstm)
    X_train = X_all[train_index]
    X_test = X_all[test_index]
    y_train = y_all[train_index]
    y_test = y_all[test_index]
    X=tf.convert_to_tensor(X_train, dtype=tf.float32)
    y=tf.convert_to_tensor(y_train, dtype=tf.float32)
    lstm.fit(X, y, batch_size=BATCH, epochs=EPOCHS)
    truth.append(int(y_test[0]))
    score = lstm.predict(X_test)[0][0]
    print("Truth:",y_test,"Score:",score)
    predicted.append(int(score+0.49999))  # use less than 1/2 to avoid exceeding 1
print("truth\n",truth)
print("predicted\n",predicted)

Num splits: 40
Epoch 1/5
4/4 [==============================] - 8s 53ms/step - loss: 0.6739 - accuracy: 0.5759
Epoch 2/5
4/4 [==============================] - 0s 38ms/step - loss: 0.5567 - accuracy: 0.8685
Epoch 3/5
4/4 [==============================] - 0s 39ms/step - loss: 0.4507 - accuracy: 0.9154
Epoch 4/5
4/4 [==============================] - 0s 41ms/step - loss: 0.3169 - accuracy: 0.9190
Epoch 5/5
4/4 [==============================] - 0s 41ms/step - loss: 0.2292 - accuracy: 0.9731
Truth: [0.] Score: 0.4206725
Epoch 1/5
4/4 [==============================] - 8s 44ms/step - loss: 0.7087 - accuracy: 0.3336
Epoch 2/5
4/4 [==============================] - 0s 43ms/step - loss: 0.6316 - accuracy: 0.7408
Epoch 3/5
4/4 [==============================] - 0s 51ms/step - loss: 0.5591 - accuracy: 0.8182
Epoch 4/5
4/4 [==============================] - 0s 39ms/step - loss: 0.4761 - accuracy: 0.8918
Epoch 5/5
4/4 [==============================] - 0s 38ms/step - loss: 0.3643 - accuracy: 0.9

Truth: [0.] Score: 0.097489774
Epoch 1/5
4/4 [==============================] - 8s 41ms/step - loss: 0.7142 - accuracy: 0.4687
Epoch 2/5
4/4 [==============================] - 0s 36ms/step - loss: 0.5617 - accuracy: 0.8082
Epoch 3/5
4/4 [==============================] - 0s 37ms/step - loss: 0.4663 - accuracy: 0.8892
Epoch 4/5
4/4 [==============================] - 0s 36ms/step - loss: 0.3180 - accuracy: 1.0000
Epoch 5/5
4/4 [==============================] - 0s 38ms/step - loss: 0.2283 - accuracy: 1.0000
Truth: [0.] Score: 0.17515269
Epoch 1/5
4/4 [==============================] - 8s 42ms/step - loss: 0.7023 - accuracy: 0.4418
Epoch 2/5
4/4 [==============================] - 0s 37ms/step - loss: 0.6035 - accuracy: 0.7310
Epoch 3/5
4/4 [==============================] - 0s 39ms/step - loss: 0.5308 - accuracy: 0.8179
Epoch 4/5
4/4 [==============================] - 0s 42ms/step - loss: 0.4356 - accuracy: 0.9056
Epoch 5/5
4/4 [==============================] - 0s 37ms/step - loss: 0.330

Truth: [0.] Score: 0.02605915
Epoch 1/5
4/4 [==============================] - 8s 42ms/step - loss: 0.6944 - accuracy: 0.5426
Epoch 2/5
4/4 [==============================] - 0s 39ms/step - loss: 0.5810 - accuracy: 0.9192
Epoch 3/5
4/4 [==============================] - 0s 35ms/step - loss: 0.4723 - accuracy: 0.9092
Epoch 4/5
4/4 [==============================] - 0s 36ms/step - loss: 0.3165 - accuracy: 0.9562
Epoch 5/5
4/4 [==============================] - 0s 35ms/step - loss: 0.2208 - accuracy: 0.9362
Truth: [0.] Score: 0.023969442
Epoch 1/5
4/4 [==============================] - 8s 42ms/step - loss: 0.6910 - accuracy: 0.5021
Epoch 2/5
4/4 [==============================] - 0s 36ms/step - loss: 0.5711 - accuracy: 0.7849
Epoch 3/5
4/4 [==============================] - 0s 36ms/step - loss: 0.4496 - accuracy: 0.9626
Epoch 4/5
4/4 [==============================] - 0s 37ms/step - loss: 0.3450 - accuracy: 0.9628
Epoch 5/5
4/4 [==============================] - 0s 36ms/step - loss: 0.240

Truth: [0.] Score: 0.38300776
Epoch 1/5
4/4 [==============================] - 8s 42ms/step - loss: 0.6681 - accuracy: 0.6672
Epoch 2/5
4/4 [==============================] - 0s 36ms/step - loss: 0.5527 - accuracy: 0.8182
Epoch 3/5
4/4 [==============================] - 0s 35ms/step - loss: 0.4770 - accuracy: 0.7682
Epoch 4/5
4/4 [==============================] - 0s 35ms/step - loss: 0.3609 - accuracy: 0.8823
Epoch 5/5
4/4 [==============================] - 0s 40ms/step - loss: 0.1984 - accuracy: 0.9692
Truth: [0.] Score: 0.024620473
Epoch 1/5
4/4 [==============================] - 8s 41ms/step - loss: 0.7126 - accuracy: 0.5026
Epoch 2/5
4/4 [==============================] - 0s 37ms/step - loss: 0.5587 - accuracy: 0.8451
Epoch 3/5
4/4 [==============================] - 0s 36ms/step - loss: 0.4432 - accuracy: 0.8823
Epoch 4/5
4/4 [==============================] - 0s 36ms/step - loss: 0.3202 - accuracy: 0.9428
Epoch 5/5
4/4 [==============================] - 0s 36ms/step - loss: 0.252

Truth: [0.] Score: 0.08372781
Epoch 1/5
4/4 [==============================] - 8s 41ms/step - loss: 0.6677 - accuracy: 0.4887
Epoch 2/5
4/4 [==============================] - 0s 38ms/step - loss: 0.5340 - accuracy: 0.8351
Epoch 3/5
4/4 [==============================] - 0s 36ms/step - loss: 0.3816 - accuracy: 0.8959
Epoch 4/5
4/4 [==============================] - 0s 40ms/step - loss: 0.2769 - accuracy: 0.9562
Epoch 5/5
4/4 [==============================] - 0s 36ms/step - loss: 0.2243 - accuracy: 1.0000
Truth: [0.] Score: 0.38970095
Epoch 1/5
4/4 [==============================] - 8s 41ms/step - loss: 0.6509 - accuracy: 0.6972
Epoch 2/5
4/4 [==============================] - 0s 38ms/step - loss: 0.5148 - accuracy: 0.9118
Epoch 3/5
4/4 [==============================] - 0s 37ms/step - loss: 0.4288 - accuracy: 0.7985
Epoch 4/5
4/4 [==============================] - 0s 36ms/step - loss: 0.2776 - accuracy: 0.9628
Epoch 5/5
4/4 [==============================] - 0s 36ms/step - loss: 0.1551

Truth: [0.] Score: 0.6039754
Epoch 1/5
4/4 [==============================] - 8s 62ms/step - loss: 0.7163 - accuracy: 0.4718
Epoch 2/5
4/4 [==============================] - 0s 61ms/step - loss: 0.5715 - accuracy: 0.8015
Epoch 3/5
4/4 [==============================] - 0s 49ms/step - loss: 0.4421 - accuracy: 0.8418
Epoch 4/5
4/4 [==============================] - 0s 51ms/step - loss: 0.3368 - accuracy: 0.8854
Epoch 5/5
4/4 [==============================] - 0s 53ms/step - loss: 0.2135 - accuracy: 0.9831
Truth: [0.] Score: 0.014749259
Epoch 1/5
4/4 [==============================] - 9s 44ms/step - loss: 0.7005 - accuracy: 0.5759
Epoch 2/5
4/4 [==============================] - 0s 42ms/step - loss: 0.6032 - accuracy: 0.7013
Epoch 3/5
4/4 [==============================] - 0s 41ms/step - loss: 0.4841 - accuracy: 0.8618
Epoch 4/5
4/4 [==============================] - 0s 38ms/step - loss: 0.3477 - accuracy: 0.9662
Epoch 5/5
4/4 [==============================] - 0s 46ms/step - loss: 0.2371

In [15]:
confusion = confusion_matrix(truth,predicted)
confusion

array([[23,  3],
       [ 7,  7]])

## Useful links
Masking and Padding with Keras  
https://www.tensorflow.org/guide/keras/masking_and_padding

Train and test with Keras  
https://www.tensorflow.org/guide/keras/train_and_evaluate
